**Instructions**


Over the past few years, ride-sharing apps have been on the rise across many cities in the world. 
While this has happened, Uber and Lyft's ride prices are not constant like
public transport. They are greatly affected by the demand and supply of rides at a given time.
As a Data Scientist working to understand this market, you have been tasked to come up with a descriptive analysis report to help a Ride-Sharing Startup coming into this space, understand the various patterns on how pricing works for the existing ride-sharing company.
Luckily, you were able to access some real-time data from Uber & Lyft's API and weather data from Weather API conditions.
You build a custom application in Scala to query data at regular intervals and saved it to DynamoDB. The queried cab ride estimates are done after every 5 mins and weather data after every 1 hr.
The cab ride data covers various types of cabs for Uber & Lyft and their price for the given location. Weather data contains weather attributes like temperature, rain, cloud,  etc for all the locations taken into consideration.



**Pre-requisites**


In [15]:
# We will first load an sql extension. 
# Notebook extensions are simple add-ons that extend the basic 
# functionality of the notebook environment. 
# This extension will allow us to run SQL code in our Notebook.
# ---
#
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [16]:
# We will then connect to an in memory SQLite database within colaboratory. 
# Something to note is that this in memory dataset will be deleted as 
# soon the dataset connection is lost, meaning when our connection to 
# the servers is disconnected then our database will cease to exist.
# ---
#  
%sql sqlite://

'Connected: @None'

In [17]:
# We will now need the pandas library to read our csv file 
# from an external the source. 
# ---
#
import pandas as pd

In [18]:
cab_rides = pd.read_csv("cab_rides.csv", encoding="ISO-8859-1")





**Importing** **Data**

In [19]:
# We load our first dataset from a csv file as shown then  
# after store the dataset in our in memory sqlite database.
# We read our dataset from its source and store it in a dataframe 
# called cab_rides 
# ---
#
cab_rides = pd.read_csv('/content/cab_rides.csv') 

# Then store it in an SQL table of our in memory sqlite database 
# --- 
# 

# We then delete the table if it exists in our database
# ---
#
%sql DROP TABLE IF EXISTS cab_rides;

# And finally store our table in the table name insurance within our dataset.
# The persist command will create a table in the database to which we are connected, 
# the table name will be the same as dataframe variable.
# ---
#
%sql PERSIST cab_rides;

# Preview the first 5 records of the insurance dataset 
# ---
#
%sql SELECT * FROM cab_rides LIMIT 5;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


index,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name
0,0.44,Lyft,1544950000000.0,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared
1,0.44,Lyft,1543280000000.0,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux
2,0.44,Lyft,1543370000000.0,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft
3,0.44,Lyft,1543550000000.0,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL
4,0.44,Lyft,1543460000000.0,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL


In [13]:
weather = pd.read_csv("/content/weather.csv", encoding="ISO-8859-1")

In [14]:
# We load our first dataset from a csv file as shown then  
# after store the dataset in our in memory sqlite database.
# We read our dataset from its source and store it in a dataframe 
# called weather
# ---
#
weather = pd.read_csv('/content/weather.csv') 

# Then store it in an SQL table of our in memory sqlite database 
# --- 
# 

# We then delete the table if it exists in our database
# ---
#
%sql DROP TABLE IF EXISTS weather;

# And finally store our table in the table name insurance within our dataset.
# The persist command will create a table in the database to which we are connected, 
# the table name will be the same as dataframe variable.
# ---
#
%sql PERSIST weather;

# Preview the first 5 records of the weather dataset 
# ---
#
%sql SELECT * FROM weather LIMIT 5;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


index,temp,location,clouds,pressure,rain,time_stamp,humidity,wind
0,42.42,Back Bay,1.0,1012.14,0.1228,1545003901,0.77,11.25
1,42.43,Beacon Hill,1.0,1012.15,0.1846,1545003901,0.76,11.32
2,42.5,Boston University,1.0,1012.15,0.1089,1545003901,0.76,11.07
3,42.11,Fenway,1.0,1012.13,0.0969,1545003901,0.77,11.09
4,43.13,Financial District,1.0,1012.14,0.1786,1545003901,0.75,11.49


In [25]:
#Filtering, Sorting, and Grouping
#Perform filtering, sorting, and grouping to explore a dataset using the SQL language

%%sql 
SELECT SUM(price) AS 'Total Sales from Financial District '
FROM cab_rides
WHERE source = 'Financial District';

 * sqlite://
Done.


Total Sales from Financial District
985375.7999999999


In [21]:
# Getting the avg weather pattern for locations
# ---

%%sql 
SELECT location, AVG(wind) FROM weather
GROUP BY location
ORDER BY AVG(wind) DESC
LIMIT 10;

 * sqlite://
Done.


location,AVG(wind)
Financial District,6.860019120458891
North End,6.8531166347992345
South Station,6.848948374760993
Haymarket Square,6.843193116634804
North Station,6.835755258126191
Theatre District,6.834302103250482
West End,6.816233269598474
Beacon Hill,6.8103250478011494
Back Bay,6.778527724665383
Northeastern University,6.749426386233272


In [23]:
#Data Formatting
#Perform data formatting of numeric data to prepare data for further analysis using the SQL language.

%%sql 
SELECT UPPER(source) AS SOURCE, price AS PRICE
FROM cab_rides 
LIMIT 5; 

 * sqlite://
Done.


SOURCE,PRICE
HAYMARKET SQUARE,5.0
HAYMARKET SQUARE,11.0
HAYMARKET SQUARE,7.0
HAYMARKET SQUARE,26.0
HAYMARKET SQUARE,9.0


In [26]:
#Basic Statistical Techniques
#Perform basic statistical techniques using COUNT, MIN, MAX, and SUM functions using the SQL language.
#AVG
%%sql 
SELECT cab_type, AVG(distance), AVG(price) FROM cab_rides
WHERE destination = 'Financial District'
GROUP BY cab_type 
ORDER BY AVG(distance) DESC
LIMIT 5;

 * sqlite://
Done.


cab_type,AVG(distance),AVG(price)
Uber,2.6406739643702153,17.15554124633326
Lyft,2.5251497827579095,18.99527403003278


In [27]:
#COUNT
%%sql 
SELECT cab_type, destination, Count(cab_type) FROM cab_rides
GROUP BY destination
ORDER BY cab_type DESC
LIMIT 10;

 * sqlite://
Done.


cab_type,destination,Count(cab_type)
Uber,Beacon Hill,57403
Uber,Financial District,58851
Uber,Haymarket Square,57764
Uber,North End,57756
Uber,North Station,57119
Uber,Northeastern University,57755
Uber,South Station,57749
Uber,West End,57575
Lyft,Back Bay,57780
Lyft,Boston University,57764


In [28]:
#SUM
%%sql 
SELECT cab_type, product_id, SUM(price) FROM cab_rides
GROUP BY product_id
ORDER BY SUM(price) DESC
LIMIT 10;



 * sqlite://
Done.


cab_type,product_id,SUM(price)
Uber,6d318bcc-22a3-4af6-bddd-b409bfce1546,1668679.5
Lyft,lyft_luxsuv,1656124.55
Lyft,lyft_lux,1181605.55
Uber,6c84fd89-3f11-4782-9b50-97c468b19529,1130758.0
Lyft,lyft_premier,910509.5
Uber,6f72dfc5-27f1-42e8-84db-ccc7a75f6969,863803.0
Lyft,lyft_plus,784375.2
Uber,9a0e7b09-b92b-4c41-9779-2ad22b4d779d,538013.5
Uber,55c66225-fbe7-4fd5-9072-eab1ece5e23e,537997.0
Lyft,lyft,492413.67999999993


In [29]:
#MIN
%%sql 
SELECT cab_type, product_id, MIN(price) FROM cab_rides
GROUP BY product_id
ORDER BY MIN(price) DESC
LIMIT 10;


 * sqlite://
Done.


cab_type,product_id,MIN(price)
Lyft,lyft_luxsuv,26.0
Uber,6d318bcc-22a3-4af6-bddd-b409bfce1546,23.0
Lyft,lyft_lux,16.5
Uber,6c84fd89-3f11-4782-9b50-97c468b19529,13.5
Lyft,lyft_premier,10.5
Lyft,lyft_plus,9.0
Uber,6f72dfc5-27f1-42e8-84db-ccc7a75f6969,8.0
Uber,55c66225-fbe7-4fd5-9072-eab1ece5e23e,6.0
Uber,9a0e7b09-b92b-4c41-9779-2ad22b4d779d,6.0
Lyft,lyft,5.0


In [30]:
#MAX
%%sql 
SELECT cab_type, product_id, MAX(price) FROM cab_rides
GROUP BY product_id
ORDER BY MAX(price) DESC
LIMIT 10;


 * sqlite://
Done.


cab_type,product_id,MAX(price)
Lyft,lyft_luxsuv,97.5
Uber,6d318bcc-22a3-4af6-bddd-b409bfce1546,89.5
Uber,6f72dfc5-27f1-42e8-84db-ccc7a75f6969,76.0
Lyft,lyft_lux,75.0
Uber,6c84fd89-3f11-4782-9b50-97c468b19529,68.5
Lyft,lyft_plus,65.0
Lyft,lyft_premier,55.0
Uber,55c66225-fbe7-4fd5-9072-eab1ece5e23e,44.0
Uber,9a0e7b09-b92b-4c41-9779-2ad22b4d779d,44.0
Uber,997acbb5-e102-41e1-b155-9df7de0a73f2,42.5


In [ ]:
%%sql
SELECT *
FROM cab_rides, weather
WHERE cab_rides.destination = weather.location
LIMIT 5;

 * sqlite://
Done.


index,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,index_1,temp,location,clouds,pressure,rain,time_stamp_1,humidity,wind
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,7,42.21,North Station,1.0,1012.16,0.2069,1545003901,0.77,11.37
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,18,41.95,North Station,0.81,991.63,None,1543347920,0.73,10.87
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,23,43.92,North Station,1.0,1006.29,0.0409,1543277833,0.9,10.09
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,43,27.22,North Station,0.15,1033.4,None,1544787901,0.81,3.01
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,54,45.47,North Station,1.0,1012.47,None,1543253710,0.86,6.5
